# Grading students' submissions

## Overall solution
The CLI interface for the autograder works only with docker containers.  
If we want to grade without containers, we will need to use other means.

In this notebook, we use the `grade_submission` API.  
But is is very limited:
* It grades single submissions, not batches => we need to automate for batch + use parallel threads for performance?
* It returns an object, that we will need to "parse" to reconstruct a moodle grading sheet (at least the CLI interface generates a CSV file... maybe it is possible to reuse part of the code?)

TODO: for the moment I tested grading on individual .ipynb files, need to test on zip files

## How to use it

* Go into the dist folder, edit the student version of the assignment and use the public tests
* Then try to grade it with the code below

In [ ]:
from otter.api import grade_submission # grading api
import glob # patterns in accessing file paths
import pandas as pd # pandas for saving as csv
pd.set_option('display.max_colwidth', None)
import csv # csv quoting options
from datetime import datetime # date formatting
from IPython.display import display # display for debug
import itertools as it # iteration tools

## Grading parameters

Assignment parameters:

In [ ]:
# Name of the assignment **file**
assignmentname = "assignment"
# Folder in which the assignment and grader has been generated
distributionfolder = "dist"

Grader parameters:

In [ ]:
# Folder in which to put the output of the grading
graderoutputfolder = "gradebook"
# Name of the CSV file in which to store the details of the grading
graderdetailsfilename = "graderdetails"
# Name of the CSV file with the overall grade (later used to fill out the moodle file)
graderresultfilename = "graderresult"

In [ ]:
# Include the messages from the test cases as feedback for the student
includefeedbackmessages = True

Submission parameters:

In [ ]:
# Folder in which to find students' submission
submissionfolder = "moodlesubmissions/"

In [ ]:
# Folder where to find the moodle grading sheets
moodlegradingsheetfolder = "gradebook"

## Grading and generation of the output

First retrieving the grader:

In [ ]:
graderzip = glob.glob(distributionfolder+"/autograder/"+assignmentname+"-autograder_*.zip")[0]
graderzip

Then retrieving students' submissions

In [ ]:
submissionlist = glob.glob(submissionfolder+"/*")
submissionlist

Iterating over submissions, calling the grader and storing the results into CSV files

**TODO**:
* Check if the CSV with the details is still readable with the line returns...

In [ ]:
# Step 1: generate 2 CSV files for each student: 1 with the overall grade and 1 with the detailed grading
for submission in submissionlist:
    
    # Call the autograder and get the result
    gradingoutput = grade_submission(submission+"/"+assignmentname+".ipynb", graderzip, quiet=True)
    
    # Gather the details of the grading
    detailedresults = [["Exercise", "Grade", "Possible", "Feedback"]]
    for exercisename, exercise in gradingoutput.results.items():
        feedbackmessage = ""
        
        # Iterate over the test results to collect the feedback messages if any
        if includefeedbackmessages:
            for test_case_result in exercise.test_case_results:
                if (test_case_result.passed and (test_case_result.test_case.success_message is not None)):
                    feedbackmessage+= test_case_result.test_case.name+": "+test_case_result.test_case.success_message+"\n"
                elif ((not test_case_result.passed) and (test_case_result.test_case.failure_message is not None)):
                    feedbackmessage+= test_case_result.test_case.name+": "+test_case_result.test_case.failure_message+"\n"
        
        detailedresults.append([exercise.name, exercise.score, exercise.possible, feedbackmessage])

    # Save the details of the grading
    graderdetailsfile = submission+"/"+graderoutputfolder+"/"+graderdetailsfilename+".csv"
    pd.DataFrame(detailedresults).to_csv(graderdetailsfile, header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)

    
    # Gather the total grade for the submission and save it
    graderresultfile = submission+"/"+graderoutputfolder+"/"+graderresultfilename+".csv"
    graderresultdf = pd.DataFrame([[submission, gradingoutput.total, gradingoutput.possible]], 
                                  columns=["Submission Name", "Grade", "Possible"])
    graderresultdf.to_csv(graderresultfile, index=False, quoting=csv.QUOTE_NONNUMERIC)    

## Filling out the CSV files that are specific to moodle

**TODO**:
* generate the feedback: detail of points + messages

In [ ]:
# Include details into overall moodle feedback
includedetailsgrades = True
includedetailsmsgs = True # NB: messages will be included only if grades are included

In [ ]:
# Step 2: fill out the individual CSV from moodle for each student
for submission in submissionlist:

    # Read the moodle file
    moodlegradebookfile = glob.glob(submission+"/"+moodlegradingsheetfolder+"/*_grading.csv")[0]
    moodledf = pd.read_csv(moodlegradebookfile)
    
    # Read the grader file
    graderresultfile = glob.glob(submission+"/"+graderoutputfolder+"/"+graderresultfilename+".csv")[0]
    graderresultdf = pd.read_csv(graderresultfile)
    
    if includedetailsgrades:
        # Read the grader details file
        graderdetailsfile = glob.glob(submission+"/"+graderoutputfolder+"/"+graderdetailsfilename+".csv")[0]
        graderdetailstdf = pd.read_csv(graderdetailsfile)
        
        # Formatters to get a pretty rendering of the info
        exerciseformatter = lambda x: 'Question %s:' % x
        gradeformatter = lambda x: '%s /' % x
        msgformatter = lambda x: '=> Messages: %s' % x.replace("\n", "")
        
        # Transform the details into string
        details = graderdetailstdf.to_string(index=False, header=False, 
                                         columns=["Exercise", "Grade", "Possible"] + (["Feedback"] if includedetailsmsgs else []),
                                         formatters={'Exercise': exerciseformatter, 'Grade': gradeformatter, 'Feedback': msgformatter})
        
        # Add the details to the moodle info
        moodledf.loc[0, "Feedback comments"] = details

        
    # Modify the moodle info
    moodledf.loc[0, "Grade"] = graderresultdf.loc[0, "Grade"]
    moodledf.loc[0, "Maximum Grade"] = graderresultdf.loc[0, "Possible"] # Actually this line is useless, it is not used by moodle
    moodledf.loc[0, "Last modified (grade)"] = datetime.today().strftime('%A, %d %B %Y %H:%M') # This line is important for the file to be read by moodle!!!
    
    
    # Write the moodle file
    moodledf.to_csv(moodlegradebookfile, index=False, quoting=csv.QUOTE_NONNUMERIC) # The quotes are important for the file to be read by moodle

In [ ]:
# Step 3: concatenate all moodle CSV files into 1 global gradebook